In [6]:
from pyspark.sql import SparkSession
from pyspark.ml.feature import VectorAssembler,StringIndexer,OneHotEncoder
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

spark = SparkSession.builder.getOrCreate()
irisDF = spark.read.option('header','true').option('inferSchema','true').csv('datasets/iris-dataset.txt')
indexer = StringIndexer(inputCol='class',outputCol='label')
indexerModel = indexer.fit(irisDF)
irisDF = indexerModel.transform(irisDF)
#Since feature 1 (Sepal-width) is the least important feautre we are not selecting it
vec = VectorAssembler(inputCols=['petal-length','petal-width'],outputCol='features')
irisDF = vec.transform(irisDF)
#irisDF.show()
irisDF =irisDF.select('features','label')
#irisDF.show()

trainDF, testDF = irisDF.randomSplit([0.75,0.25],seed=1234)

rfClassifier = RandomForestClassifier()
model = rfClassifier.fit(trainDF)

resultDF = model.transform(testDF)
eva = MulticlassClassificationEvaluator(metricName='accuracy')
accuracy = eva.evaluate(resultDF)
print("Accuracy : ",accuracy)

#model = rfClassifier.fit(irisDF)
#print(model.featureImportances)

Accuracy :  0.9318181818181818
